<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.1.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [8]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [9]:
!rm -rf logs

In [10]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 19s 737ms/step - loss: 0.6733 - accuracy: 0.5791 - val_loss: 0.6457 - val_accuracy: 0.6244
Epoch 2/10
26/26 [==============================] - 19s 715ms/step - loss: 0.6214 - accuracy: 0.6655 - val_loss: 0.7163 - val_accuracy: 0.4780
Epoch 3/10
26/26 [==============================] - 18s 704ms/step - loss: 0.5652 - accuracy: 0.7372 - val_loss: 0.6104 - val_accuracy: 0.6390
Epoch 4/10
26/26 [==============================] - 18s 701ms/step - loss: 0.5269 - accuracy: 0.7530 - val_loss: 0.4786 - val_accuracy: 0.8049
Epoch 5/10
26/26 [==============================] - 18s 695ms/step - loss: 0.4568 - accuracy: 0.8054 - val_loss: 0.4606 - val_accuracy: 0.8195
Epoch 6/10
26/26 [==============================] - 18s 692ms/step - loss: 0.4321 - accuracy: 0.7810 - val_loss: 0.3883 - val_accuracy: 0.8439
Epoch 7/10
26/26 [==============================] - 18s 704ms/step - loss: 0.3743 - accuracy: 0.8321 - val_loss: 0.3454 - val_accuracy: 0.8976

In [11]:
%tensorboard --logdir logs

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [12]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 00001: saving model to weights.01-0.65.h5
26/26 - 19s - loss: 0.6719 - accuracy: 0.5669 - val_loss: 0.6470 - val_accuracy: 0.6244
Epoch 2/5

Epoch 00002: saving model to weights.02-0.59.h5
26/26 - 18s - loss: 0.6195 - accuracy: 0.7165 - val_loss: 0.5858 - val_accuracy: 0.7561
Epoch 3/5

Epoch 00003: saving model to weights.03-0.52.h5
26/26 - 32s - loss: 0.5679 - accuracy: 0.7397 - val_loss: 0.5195 - val_accuracy: 0.7805
Epoch 4/5

Epoch 00004: saving model to weights.04-0.52.h5
26/26 - 18s - loss: 0.5371 - accuracy: 0.7263 - val_loss: 0.5239 - val_accuracy: 0.7463
Epoch 5/5

Epoch 00005: saving model to weights.05-0.41.h5
26/26 - 19s - loss: 0.4499 - accuracy: 0.8273 - val_loss: 0.4100 - val_accuracy: 0.8683


In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 00001: saving model to saved_model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


26/26 - 19s - loss: 0.6706 - accuracy: 0.5998 - val_loss: 0.6179 - val_accuracy: 0.8146


In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 00001: saving model to model.h5
26/26 - 19s - loss: 0.6773 - accuracy: 0.5620 - val_loss: 0.6805 - val_accuracy: 0.4439
Epoch 2/2

Epoch 00002: saving model to model.h5
26/26 - 18s - loss: 0.6471 - accuracy: 0.6131 - val_loss: 0.6361 - val_accuracy: 0.7415


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 19s - loss: 0.6653 - accuracy: 0.5961 - val_loss: 0.6534 - val_accuracy: 0.5171
Epoch 2/50
26/26 - 19s - loss: 0.6131 - accuracy: 0.6959 - val_loss: 0.5620 - val_accuracy: 0.8195
Epoch 3/50
26/26 - 18s - loss: 0.5514 - accuracy: 0.7567 - val_loss: 0.5327 - val_accuracy: 0.7610
Epoch 4/50
26/26 - 18s - loss: 0.4825 - accuracy: 0.7871 - val_loss: 0.4339 - val_accuracy: 0.8439
Epoch 5/50
26/26 - 18s - loss: 0.4111 - accuracy: 0.8491 - val_loss: 0.3359 - val_accuracy: 0.9122
Epoch 6/50
26/26 - 18s - loss: 0.3544 - accuracy: 0.8613 - val_loss: 0.2785 - val_accuracy: 0.9268
Epoch 7/50
26/26 - 19s - loss: 0.2895 - accuracy: 0.8820 - val_loss: 0.2185 - val_accuracy: 0.9610
Epoch 8/50
26/26 - 18s - loss: 0.2862 - accuracy: 0.8954 - val_loss: 0.2122 - val_accuracy: 0.9610
Epoch 9/50
26/26 - 19s - loss: 0.1962 - accuracy: 0.9562 - val_loss: 0.1430 - val_accuracy: 0.9805
Epoch 10/50
26/26 - 18s - loss: 0.1620 - accuracy: 0.9574 - val_loss: 0.1388 - val_accuracy: 0.9561
Epoch 11/

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 19s 715ms/step - loss: 0.6550 - accuracy: 0.6229 - val_loss: 0.6226 - val_accuracy: 0.7463
Epoch 2/5
26/26 [==============================] - 18s 709ms/step - loss: 0.5895 - accuracy: 0.7153 - val_loss: 0.5870 - val_accuracy: 0.6341
Epoch 3/5
26/26 [==============================] - 19s 715ms/step - loss: 0.5370 - accuracy: 0.7263 - val_loss: 0.5064 - val_accuracy: 0.8098
Epoch 4/5
26/26 [==============================] - 18s 696ms/step - loss: 0.4735 - accuracy: 0.7786 - val_loss: 0.4399 - val_accuracy: 0.8049
Epoch 5/5
26/26 [==============================] - 18s 704ms/step - loss: 0.4268 - accuracy: 0.8151 - val_loss: 0.3765 - val_accuracy: 0.8146


In [17]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.622871,0.655803,0.746341,0.622588
1,1,0.715328,0.589848,0.634146,0.586960
2,2,0.726277,0.537930,0.809756,0.506384
3,3,0.778589,0.475310,0.804878,0.439857
4,4,0.815085,0.428135,0.814634,0.376526


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [18]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 26s 994ms/step - loss: 0.6813 - accuracy: 0.5438 - val_loss: 0.6868 - val_accuracy: 0.4341

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 18s 707ms/step - loss: 0.6701 - accuracy: 0.5377 - val_loss: 0.6820 - val_accuracy: 0.4341

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 19s 727ms/step - loss: 0.6654 - accuracy: 0.5462 - val_loss: 0.6799 - val_accuracy: 0.4341

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 18s 708ms/step - loss: 0.6631 - accuracy: 0.5401 - val_loss: 0.6781 - val_accuracy: 0.4488

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 18s 692ms/step - loss: 0.6

In [19]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [ ]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 21s 795ms/step - loss: 0.6655 - accuracy: 0.6095 - val_loss: 0.6764 - val_accuracy: 0.4732
Epoch 2/50
26/26 [==============================] - 19s 714ms/step - loss: 0.6220 - accuracy: 0.6861 - val_loss: 0.5886 - val_accuracy: 0.7415
Epoch 3/50
26/26 [==============================] - 19s 735ms/step - loss: 0.5558 - accuracy: 0.7616 - val_loss: 0.5429 - val_accuracy: 0.7756
Epoch 4/50
26/26 [==============================] - 19s 719ms/step - loss: 0.4868 - accuracy: 0.7932 - val_loss: 0.4803 - val_accuracy: 0.8146
Epoch 5/50
26/26 [==============================] - 19s 719ms/step - loss: 0.4485 - accuracy: 0.8139 - val_loss: 0.4100 - val_accuracy: 0.8537
Epoch 6/50
26/26 [==============================] - 19s 715ms/step - loss: 0.3890 - accuracy: 0.8345 - val_loss: 0.3868 - val_accuracy: 0.8585
Epoch 7/50
26/26 [==============================] - 18s 704ms/step - loss: 0.3462 - accuracy: 0.8710 - val_loss: 0.2623 - val_accuracy: 0.9268

In [ ]:
%tensorboard --logdir log_dir